In [6]:
# Cell 1: Imports & config

import pandas as pd
import numpy as np

from collections import Counter
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)


DATA_PATH = "/Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/predictive_maintenance.csv"  # <-- change per dataset
DATA_FORMAT = "csv"                    # "csv" or "parquet"

# Thresholds 
COL_MISSING_DROP_THRESHOLD = 0.30   # drop cols with >30% NaN
ROW_MISSING_DROP_THRESHOLD = 0.50   # optional: drop rows with >50% NaN
WINSOR_LIMITS = (0.01, 0.01)        # 1% winsorization on each tail
HIGH_CORR_THRESHOLD = 0.98          # consider >0.98 as duplicate


In [7]:
# Cell 2: Load dataset

if DATA_FORMAT == "csv":
    df = pd.read_csv(DATA_PATH)
elif DATA_FORMAT == "parquet":
    df = pd.read_parquet(DATA_PATH)
else:
    raise ValueError("Unsupported DATA_FORMAT. Use 'csv' or 'parquet'.")

print("Loaded dataset with shape:", df.shape)
df.head()


Loaded dataset with shape: (10000, 12)


,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [8]:
# Cell 3: Basic dataset overview

n_rows, n_cols = df.shape
print(f"Rows: {n_rows}, Columns: {n_cols}\n")

print("Column dtypes:")
display(df.dtypes)

print("\nMissing values per column:")
missing_counts = df.isnull().sum()
display(missing_counts)

print("\nPercentage of missing values per column:")
missing_pct = (df.isnull().mean() * 100).round(2)
display(missing_pct)

print("\nDescriptive stats for numeric columns:")
display(df.describe(include=[np.number]).T)

print("\nQuick checks:")
print("Duplicate rows:", df.duplicated().sum())
print("Constant columns:", (df.nunique(dropna=False) <= 1).sum())


Rows: 10000, Columns: 12

Column dtypes:


Type                    object
Air temperature        float64
Process temperature    float64
Rotational speed         int64
Torque                 float64
Tool wear                int64
Machine failure          int64
TWF                      int64
HDF                      int64
PWF                      int64
OSF                      int64
RNF                      int64
dtype: object


Missing values per column:


Type                   0
Air temperature        0
Process temperature    0
Rotational speed       0
Torque                 0
Tool wear              0
Machine failure        0
TWF                    0
HDF                    0
PWF                    0
OSF                    0
RNF                    0
dtype: int64


Percentage of missing values per column:


Type                   0.0
Air temperature        0.0
Process temperature    0.0
Rotational speed       0.0
Torque                 0.0
Tool wear              0.0
Machine failure        0.0
TWF                    0.0
HDF                    0.0
PWF                    0.0
OSF                    0.0
RNF                    0.0
dtype: float64


Descriptive stats for numeric columns:


,count,mean,std,min,25%,50%,75%,max
Air temperature,10000.0,300.00493,2.000259,295.3,298.3,300.1,301.5,304.5
Process temperature,10000.0,310.00556,1.483734,305.7,308.8,310.1,311.1,313.8
Rotational speed,10000.0,1538.77610,179.284096,1168.0,1423.0,1503.0,1612.0,2886.0
Torque,10000.0,39.98691,9.968934,3.8,33.2,40.1,46.8,76.6
Tool wear,10000.0,107.95100,63.654147,0.0,53.0,108.0,162.0,253.0
Machine failure,10000.0,0.03390,0.180981,0.0,0.0,0.0,0.0,1.0
TWF,10000.0,0.00460,0.067671,0.0,0.0,0.0,0.0,1.0
HDF,10000.0,0.01150,0.106625,0.0,0.0,0.0,0.0,1.0
PWF,10000.0,0.00950,0.097009,0.0,0.0,0.0,0.0,1.0
OSF,10000.0,0.00980,0.098514,0.0,0.0,0.0,0.0,1.0



Quick checks:
Duplicate rows: 0
Constant columns: 0


In [16]:
# Cell 4: Variable type detection

def detect_variable_type(
    s: pd.Series,
    n_rows: int,
    cat_threshold: int = 20,
    cat_ratio_threshold: float = 0.01
):
    """
    Heuristic:
    - If only one unique value -> 'constant'
    - If only {0,1} or {True,False} -> 'binary'
    - If non-numeric -> 'categorical'
    - If numeric:
        * if low unique and low unique ratio -> 'categorical' (code-like)
        * else if unique <= cat_threshold -> 'discrete_numeric'
        * else -> 'continuous'
    """
    non_na = s.dropna()
    if non_na.empty:
        return "constant"

    unique_vals = pd.unique(non_na)
    n_unique = len(unique_vals)

    if n_unique <= 1:
        return "constant"

    # Normalize for binary check (handles bool, int, float, and "0"/"1")
    try:
        as_num = pd.to_numeric(non_na, errors="coerce").dropna()
        unique_num = set(as_num.unique())
    except Exception:
        unique_num = set()

    if unique_num and unique_num.issubset({0, 1}):
        return "binary"
    if set(non_na.unique()).issubset({True, False}):
        return "binary"

    # Non-numeric -> categorical
    if not pd.api.types.is_numeric_dtype(s):
        return "categorical"

    # Numeric: decide categorical vs discrete_numeric vs continuous
    unique_ratio = n_unique / max(n_rows, 1)

    # If it looks like an ID/code: low cardinality AND very low unique ratio
    if n_unique <= cat_threshold and unique_ratio <= cat_ratio_threshold:
        return "categorical"

    if n_unique <= cat_threshold:
        return "discrete_numeric"

    return "continuous"


var_types = {}
n_unique_map = {}

n_rows = df.shape[0]

for col in df.columns:
    vtype = detect_variable_type(df[col], n_rows=n_rows, cat_threshold=20, cat_ratio_threshold=0.01)
    var_types[col] = vtype
    n_unique_map[col] = df[col].nunique(dropna=True)

var_types_series = pd.Series(var_types, name="var_type")
unique_counts_series = pd.Series(n_unique_map, name="n_unique")

summary_types = pd.concat([var_types_series, unique_counts_series, df.dtypes.rename("dtype")], axis=1)
summary_types.sort_index()

type_counts = Counter(var_types.values())
print("Original variable type counts:", dict(type_counts))



Original variable type counts: {'categorical': 1, 'continuous': 5, 'binary': 6}


In [10]:
# Cell 5: Cleaning – missing values, constant columns, high-NA columns

df_clean = df.copy()

#1 Drop columns with too many NaNs
col_missing_frac = df_clean.isnull().mean()
high_na_cols = col_missing_frac[col_missing_frac > COL_MISSING_DROP_THRESHOLD].index.tolist()

print(f"Columns with >{int(COL_MISSING_DROP_THRESHOLD*100)}% NaN (will be dropped):", high_na_cols)
df_clean.drop(columns=high_na_cols, inplace=True)

#2 Drop constant columns (based on actual data right now)
constant_cols = [col for col in df_clean.columns if df_clean[col].nunique(dropna=False) <= 1]
print("Constant columns (will be dropped):", constant_cols)
df_clean.drop(columns=constant_cols, inplace=True)

#3 Optionally: drop rows with too many NaNs
row_missing_frac = df_clean.isnull().mean(axis=1)
rows_before = len(df_clean)
df_clean = df_clean.loc[row_missing_frac <= ROW_MISSING_DROP_THRESHOLD].copy()
rows_after = len(df_clean)

print(f"Rows before row-drop: {rows_before}, after row-drop: {rows_after}")

#4 Recompute variable types after drops (important)
var_types_clean = {}
for col in df_clean.columns:
    var_types_clean[col] = detect_variable_type(df_clean[col], n_rows=len(df_clean), cat_threshold=20, cat_ratio_threshold=0.01)

#5 Impute remaining missing values
for col in df_clean.columns:
    col_type = var_types_clean[col]

    if df_clean[col].isnull().any():
        if col_type in ("continuous", "discrete_numeric"):
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)

        elif col_type == "binary":
            # safest default for binary
            df_clean[col] = df_clean[col].fillna(0)

        else:
            # categorical: NEVER fill with 0
            mode_val = df_clean[col].mode(dropna=True)
            fill_val = mode_val.iloc[0] if not mode_val.empty else "Missing"
            df_clean[col] = df_clean[col].fillna(fill_val)

print("\nRemaining NaNs after imputation:", df_clean.isnull().sum().sum())


Columns with >30% NaN (will be dropped): []
Constant columns (will be dropped): []
Rows before row-drop: 10000, after row-drop: 10000

Remaining NaNs after imputation: 0


In [11]:
# Cell 6: Winsorize + Scale continuous variables

continuous_cols = [col for col, t in var_types_clean.items()
                   if t == "continuous" and col in df_clean.columns]

print("Continuous columns:", continuous_cols)

if len(continuous_cols) == 0:
    print("No continuous columns detected -> skipping winsorization + scaling.")
else:
    # Winsorize
    for col in continuous_cols:
        col_values = df_clean[col].astype(float).values
        df_clean[col] = np.asarray(winsorize(col_values, limits=WINSOR_LIMITS), dtype=float)

    # Scale
    scaler = StandardScaler()
    df_clean[continuous_cols] = scaler.fit_transform(df_clean[continuous_cols])

    # Keep scaler info for later 
    scaler_info = {
        "columns": continuous_cols,
        "mean": scaler.mean_.tolist(),
        "scale": scaler.scale_.tolist()
    }

    print("Finished winsorization + scaling for continuous columns.")


Continuous columns: ['Air temperature', 'Process temperature', 'Rotational speed', 'Torque', 'Tool wear']
Finished winsorization + scaling for continuous columns.


In [13]:
# Cell 7: Drop highly correlated continuous features (near duplicates)

# Only on continuous columns that survived cleaning
continuous_cols_clean = [c for c in continuous_cols if c in df_clean.columns]

if len(continuous_cols_clean) >= 2:
    corr_matrix = df_clean[continuous_cols_clean].corr().abs().fillna(0)

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Columns to drop
    to_drop_corr = [col for col in upper.columns if (upper[col] > HIGH_CORR_THRESHOLD).any()]

    print(f"Highly correlated columns to drop (>|{HIGH_CORR_THRESHOLD}|):", to_drop_corr)

    # Log the actual pairs 
    corr_pairs = []
    for col in upper.columns:
        high_corr_with = upper.index[upper[col] > HIGH_CORR_THRESHOLD].tolist()
        for other in high_corr_with:
            corr_pairs.append((other, col, float(upper.loc[other, col])))

    if len(corr_pairs) > 0:
        corr_pairs_df = pd.DataFrame(corr_pairs, columns=["col_A", "col_B", "abs_corr"]).sort_values("abs_corr", ascending=False)
        display(corr_pairs_df)

    df_clean.drop(columns=to_drop_corr, inplace=True)
else:
    print("Not enough continuous columns for correlation-based pruning.")


Highly correlated columns to drop (>|0.98|): []


In [19]:
# Cell 8: Save cleaned data 

clean_path_parquet = "/Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/cleaned_predictive_maintenance.parquet"
clean_path_csv = "/Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/cleaned_predictive_maintenance.csv"

df_clean.to_parquet(clean_path_parquet, index=False)
df_clean.to_csv(clean_path_csv, index=False)

print(f"Saved cleaned data to:\n  {clean_path_parquet}\n  {clean_path_csv}")

step1_info = {
    "original_shape": (n_rows, n_cols),
    "cleaned_shape": df_clean.shape,
    "var_types_original": var_types,
    "n_unique_original": n_unique_map,
    "type_counts_original": dict(type_counts),
}

import json
summary_path = "/Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/step1_variable_summary.json"
with open(summary_path, "w") as f:
    json.dump(step1_info, f, indent=2)
print("Saved", summary_path)



Saved cleaned data to:
  /Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/cleaned_predictive_maintenance.parquet
  /Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/cleaned_predictive_maintenance.csv
Saved /Users/pouriyasyd/Desktop/AP2 - Seyedpouria Seyedi/Data/step1_variable_summary.json
